## Audio fingerprinting
In this lab, you will practice with the core parts of the audio fingerprinting algorithm presented in http://www.ismir2002.ismir.net/proceedings/02-FP04-2.pdf.
You will see how to fingerprint a song, and then will index a collection of about 100 songs.
You will then have to implement a song retrieval algorithm employing these fingerprints, and match a number of 'unknown' song excerpts to the collection.

## What to hand in
We give you five 'unknown' excerpts. Your task is to employ the fingerprinting algorithm to identify to what files in your indexed dataset these excerpts correspond.

As final deliverable to demonstrate your successful completion of this assignment, please submit a file named [studentNumberMember1_studentNumberMember2.pdf] through Brightspace with five lines of text, one for each excerpt.

Each line should have the format "<code>excerpt_file_name matched_file_name_in_your_indexed_dataset</code>". The two filenames on each line are to be separated by a space.

For example, a line could look like "<code>excerpt-A.mp3 112233445.mp3</code>". 

## Getting started
As usual, we first import relevant libraries.

In [ ]:
import numpy as np

import librosa

from cvtools import ipynb_show_matrix
from datasets import CS4065_Dataset

%matplotlib inline

import IPython

We now fetch and load the dataset.

In [ ]:
# If you want to override the location where the dataset is stored (because you are not using the VM) 
# input the path as follows:
# cs4065_dataset = CS4065_Dataset(os.path.join(...))
cs4065_dataset = CS4065_Dataset()

songretrieval_dataset = cs4065_dataset.get_songretrieval_subset()
songretrieval_dataset_size = len(songretrieval_dataset)
print('number of songs: {}'.format(songretrieval_dataset_size))

Now, let's have a look at a random audio file.

In [ ]:
audio_file_index = np.random.randint(0, songretrieval_dataset_size)
audio_file_keys = list(songretrieval_dataset.keys())
audio_file_key = audio_file_keys[audio_file_index]  # File name.
audio_file_path = songretrieval_dataset[audio_file_key]  # File path.
print('random audio file: <{}>'.format(audio_file_key))

signal, sample_rate = librosa.core.load(audio_file_path)
assert len(np.shape(signal)) == 1, 'single channel expected'
assert sample_rate == 22050, 'unexpected sample rate found'
print('number of samples: {}'.format(len(signal)))
print('sample rate: {}'.format(sample_rate))
print('duration: {0:.1f} seconds'.format(float(len(signal)) / float(sample_rate)))

We also can render an audio player to listen to the file.

In [ ]:
IPython.display.display(IPython.display.Audio(audio_file_path))

## Fingerprint extraction
Given a music signal, we want to summarize the auditory information that it contains. The summary has to be in the form of a collection of *fingerprints*. The computed fingerprints are meant to be used both at the *indexing* and *retrieval* step.

When indexing, the fingerprints are stored in a data structure that is efficient for searching. This is because at the retrieval step, the fingerprints extracted from the excerpt to be recognized have to be matched against the indexed ones. A function that analyzes the found matches will finally determine whether the given excerpt corresponds to one of the indexed songs.

## Pipeline
Assuming that the input signal has a single channel (i.e., mono) and its sample rate is 22050 Hz, fingerpints are extracted using the following pipeline:
 1. Apply a short-time Fourier Transform (STFT) (no overlap, 512 samples per frame) to retrieve the frequencies present in the signal
 2. Compute the magnitude spectrum of the signal, representing the amount of energy for each of the frequencies
 3. Applying the perceptual Bark scale (17 bands), compute the magnitude per band
 4. Compute first-order derivatives in the frequency and time domain
 5. Apply binarization to efficiently encode the derivatives.

We will assist you in building the steps to this pipeline: see the blocks below. Upon setting up the pipeline, you will be asked to use it for indexing and retrieval.

### Short-time Fourier Transform (STFT)
We represent the input signal in the frequency domain over time. To this end, we divide the signal in non-overlapping frames of 512 samples each. For each frame, we compute the FFT. The output is a matrix of spectral coefficients.

In [ ]:
def compute_stft(signal, frame_size = 512):
    print('computing STFT')
    signal = np.array(signal)
    assert len(np.shape(signal)) == 1, 'single channel expected'
    number_of_samples = len(signal)
    number_of_full_frames = int(np.floor(float(number_of_samples) / float(frame_size)))
    print(' - number of full frames: {}'.format(number_of_full_frames))

    # Remove last frame (if not full).
    print(' - original number of samples: {}'.format(number_of_samples))
    number_of_samples = number_of_full_frames * frame_size
    print(' - number of retained samples: {}'.format(number_of_samples))
    signal = signal[:number_of_samples]
    
    # Split into frames (reshape to a matrix that has one row per frame).
    signal_frames = np.reshape(signal, (number_of_full_frames, frame_size))

    # Compute frame-wise FFT.
    signal_stft = np.fft.fft(signal_frames).T
    print(' - STFT matrix size: {} x {}'.format(*np.shape(signal_stft)))

    return signal_stft

In [ ]:
# We now use this newly created function to compute the STFT
signal_stft = compute_stft(signal)

# Each row in signal_stft, that is a frequency bin, is associated to a frequency.
# We can use numpy to get the frequencies of each bin.
number_of_frequency_bins = np.shape(signal_stft)[0]
frequencies = np.fft.fftfreq(number_of_frequency_bins, d=1.0 / float(sample_rate))

# If we look at the first bin, we see that:
# - the frequency is zero,
# - all the values are real (the imaginary part is zero).
print('first bin frequency: {0:.1f} hz'.format(frequencies[0]))
print(signal_stft[0, :10])  # We only show the first 10 frames (but it holds for all the frames).

### Magnitude spectrum
For each frame, we have 512 frequency domain coefficients. The first half of each vector of spectral coefficients is the conjugate of the second half. We retain only the first half and compute the magnitudes.

In [ ]:
# First step: compute the magnitude spectrum.

def compute_magnitude_spectrum(signal_stft):
    # Throw away redundant coefficients.
    signal_stft = signal_stft[:257, :]
    
    # Flip matrix in order to have the highest frequency associated to
    # the lowest row indexes (useful for visualization).
    signal_stft = np.flipud(signal_stft)

    # Compute the magnitude: module of each coefficient (note that we have complex coefficients).
    return np.log(np.maximum(np.real(signal_stft * np.conjugate(signal_stft)), 1e-10))


In [ ]:
# Compute the magnitude spectrum
magnitude_spectrum = compute_magnitude_spectrum(signal_stft)
print('magnitude spectrum matrix: {} x {}'.format(*np.shape(magnitude_spectrum)))
ipynb_show_matrix(magnitude_spectrum, 'Magnitude spectrum', figsize=(20, 5))

### Band magnitudes
We sum up the individual magnitudes per band using 17 given perceptual bands. The latter have been pre-determined using the Bark scale (see https://en.wikipedia.org/wiki/Bark_scale).

In [ ]:
# Bark scale band boundaries.
# Each band includes the indexes in the interval [index0, index1)
# where index0 and index1 are adjacent values.
BARK_SCALE_BANDS = [0, 2, 4, 7, 10, 13, 17, 21, 26, 32, 40, 50, 62, 79, 104, 136, 184, 257]

def compute_band_magnitudes(magnitude_spectrum):
    number_of_frames = np.shape(magnitude_spectrum)[1]
    number_of_bands = len(BARK_SCALE_BANDS) - 1
    band_magnitudes = np.zeros((number_of_bands, number_of_frames))
    for band_index in range(number_of_bands):
        index0 = BARK_SCALE_BANDS[band_index]
        index1 = BARK_SCALE_BANDS[band_index + 1]
        band_magnitudes[band_index, :] = np.sum(magnitude_spectrum[index0:index1, :], axis=0)

    return band_magnitudes

In [ ]:
# We now compute the band magnitudes
band_magnitudes = compute_band_magnitudes(magnitude_spectrum)
print('band magnitudes matrix: {} x {}'.format(*np.shape(band_magnitudes)))
ipynb_show_matrix(band_magnitudes, 'Band magnitudes', figsize=(20, 5))

### Frequency and time domain derivatives
The absolute magnitude values computed above are not invariant to power level and additive noise. However, if we consider local differences along both time and frequency, we may expect to observe similar values when an excerpt has to be recognized.

In [ ]:
def compute_band_magnitudes_derivatives(band_magnitudes):    
    # Differences between neighboring bands.
    diff_frequency = band_magnitudes[1:, :] - band_magnitudes[:-1, :]

    # Differences between neighboring frames.
    diff_time = diff_frequency[:, 1:] - diff_frequency[:, :-1]

    number_of_bands, number_of_frames = np.shape(band_magnitudes)
    assert np.shape(diff_time) == (number_of_bands - 1, number_of_frames - 1)
    return diff_time

In [ ]:
# We now take the derivatives over frequency and time
band_magnitudes_diff = compute_band_magnitudes_derivatives(band_magnitudes)
print('band magnitudes derivatives matrix: {} x {}'.format(*np.shape(band_magnitudes_diff)))
ipynb_show_matrix(band_magnitudes_diff, 'Band magnitudes (derivatives)', figsize=(20, 5))

### Binarization
Finally, we further improve the robustness of the fingeprints (while reducing the amount of encoded data), by binarizing the matrix computed using <code>compute_band_magnitudes_derivatives()</code>. We simply set a zero when a value is negative, and one otherwise.

In [ ]:
def binarize(band_magnitudes_diff):
    binarized = np.ones(np.shape(band_magnitudes_diff), dtype=np.uint8)
    binarized[band_magnitudes_diff < 0.0] = 0
    return binarized

In [ ]:
# computed the binarization
band_magnitudes_diff_binarized = binarize(band_magnitudes_diff)
ipynb_show_matrix(band_magnitudes_diff_binarized, 'Band magnitudes (binarized derivatives)', figsize=(20, 5))

## Question to you: index the collection of songs
We are now ready to index the given songs. You have to iterate over the collection (see <code>songretrieval_dataset</code> above) and use the pipeline implemented above to produce a matrix of binarized band magnitude differences for each song. Store each matrix as numpy file using <code>np.save()</code> (see http://docs.scipy.org/doc/numpy-1.10.0/reference/generated/numpy.save.html). You can store the files in the same directory where the MP3 files are (e.g., use the same file name and use .npy as extension).

In [ ]:
# TODO: put your indexing code here.

## Question to you: recognize an excerpt
Assume that you are given 15 seconds long excerpts to recognize. As done at the indexing step, you have to extract the matrix of binarized magnitude differences computed by <code>binarize</code>. Then, you compare this matrix against *all* the matrices that you have stored as .npy files at the indexing step.

The result of each comparison has to be a scalar value named *Bit Error Rate* (BER).
The BER is simply computed as the number of differences between two binarized matrices (one belonging to the query excerpts and one to a candidate match) divided by the number of bits (i.e., the matrix size). When the query excerpt is shorter in duration than an indexed song, slice the indexed matrix accordingly.

In [ ]:
# TODO: put your code for recognizing an excerpt here.
# It is advisable to also create a helper function for computing the Bit Error Rate between two binarized matrices.

# Assignment
We give you five 'unknown' excerpts. Your task is to employ the fingerprinting algorithm to identify to what files in your indexed dataset these excerpts correspond.

Please submit a text file named [studentNumberMember1_studentNumberMember2.pdf] through Brightspace with five lines of text, one for each excerpt.

Each line should have the format "<code>excerpt_file_name matched_file_name_in_your_indexed_dataset</code>". The two filenames on each line are to be separated by a space.

For example, a line could look like "<code>excerpt-A.mp3 112233445.mp3</code>". 

Fetch the 'unknown' query excerpts.

In [ ]:
songretrieval_queries = cs4065_dataset.get_songretrieval_queries()

If you'd like to listen to them, you can render them all in a loop:

In [ ]:
for snippet in songretrieval_queries:
    print('%s:'.format(snippet))
    IPython.display.display(IPython.display.Audio(songretrieval_queries[snippet]))

Use your code above to fingerprint and match these excerpts to the right songs.